In [ ]:
import pandas as pd
import numpy as np
import Recommenders as Recommenders

Dataset Cleaning and EDA

In [2]:
song_df_1 = pd.read_csv('triplets_file.csv')
song_df_1.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1


In [3]:
song_df_2 = pd.read_csv('song_data.csv')
song_df_2.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [4]:
song_df_2.shape

(1000000, 5)

In [5]:
# combine both data
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on='song_id', how='left')
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [6]:
print(len(song_df_1), len(song_df_2))

2000000 1000000


In [7]:
song_df.shape

(2000000, 7)

In [8]:
# creating new feature combining title and artist name
song_df['song'] = song_df['title']+' - '+song_df['artist_name']
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly - Foo Fighters


In [10]:
# cummulative sum of listen count of the songs
song_grouped = song_df.groupby(['song']).agg({'listen_count':'count'}).reset_index()
song_grouped.head()

,song,listen_count
0,#40 - DAVE MATTHEWS BAND,1
1,& Down - Boys Noize,4
2,'97 Bonnie & Clyde - Eminem,2
3,'Round Midnight - Miles Davis,3
4,'Till I Collapse - Eminem / Nate Dogg,6


In [11]:
#Finding avg
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage'] = (song_grouped['listen_count'] / grouped_sum ) * 100
song_grouped.sort_values(['listen_count', 'song'], ascending=[0,1])

,song,listen_count,percentage
3660,Sehr kosmisch - Harmonia,45,0.45
4678,Undo - Björk,32,0.32
5105,You're The One - Dwight Yoakam,32,0.32
1071,Dog Days Are Over (Radio Edit) - Florence + Th...,28,0.28
3655,Secrets - OneRepublic,28,0.28
...,...,...,...
5139,high fives - Four Tet,1,0.01
5140,in white rooms - Booka Shade,1,0.01
5143,paranoid android - Christopher O'Riley,1,0.01
5149,¿Lo Ves? [Piano Y Voz] - Alejandro Sanz,1,0.01


Popularity Based Recommender based on score calculated from listen count

In [12]:
pr = Recommenders.popularity_recommender_py()

In [13]:
pr.create(song_df, 'user_id', 'song')

In [14]:
# display the top 10 popular songs
pr.recommend(song_df['user_id'][10])

,user_id,song,score,Rank
3660,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sehr kosmisch - Harmonia,45,1.0
4678,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,32,2.0
5105,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You're The One - Dwight Yoakam,32,3.0
1071,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,28,4.0
3655,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Secrets - OneRepublic,28,5.0
4378,b80344d063b5ccb3212f76538f3d9e43d87dca9e,The Scientist - Coldplay,27,6.0
4712,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Use Somebody - Kings Of Leon,27,7.0
3476,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Revelry - Kings Of Leon,26,8.0
1387,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Fireflies - Charttraxx Karaoke,24,9.0
1862,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Horn Concerto No. 4 in E flat K495: II. Romanc...,23,10.0


User-Item similarity algorithm operating based on co-occurance matrix

In [16]:

ir = Recommenders.item_similarity_recommender_py()
ir.create(song_df, 'user_id', 'song')

In [68]:
user_items = ir.get_user_items(song_df['user_id'][87])

In [69]:
# display user songs history
for user_item in user_items:
    print(user_item)

Somebody To Love - Justin Bieber
Ghosts 'n' Stuff (Original Instrumental Mix) - Deadmau5
My Dad's Gone Crazy - Eminem / Hailie Jade
Missing You - John Waite
Ya Nada Queda - Kudai
The Real Slim Shady - Eminem
Forgive Me - Leona Lewis
Say My Name - Destiny's Child
Just Lose It - Eminem
16 Candles - The Crests
Without Me - Eminem
Push It - Salt-N-Pepa
Speechless - Lady GaGa


In [43]:
# give song recommendation for that user
inputContent = ir.recommend(song_df['user_id'][87])
inputContent

No. of unique songs for the user: 13
no. of unique songs in the training set: 5151
Non zero values in cooccurence_matrix :2813


,user_id,song,score,rank
0,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Superman - Eminem / Dina Rae,0.096418,1
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Mockingbird - Eminem,0.089910,2
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,U Smile - Justin Bieber,0.061363,3
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm Back - Eminem,0.057932,4
4,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Here Without You - 3 Doors Down,0.055542,5
5,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Teach Me How To Dougie - California Swag District,0.054701,6
6,4bd88bfb25263a75bbdd467e74018f4ae570e5df,American Idiot [feat. Green Day & The Cast Of ...,0.054508,7
7,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Monster - Lady GaGa,0.052736,8
8,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hellbound - J-Black & Masta Ace,0.052564,9
9,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You Found Me (Album Version) - The Fray,0.052564,10


In [44]:
inputContent.shape

(10, 4)

In [45]:
songIDcol = song_df[['song_id','song']]

Appending song_id to the output of collaborative filtering algorithm

In [46]:
inputContent = pd.merge(inputContent, songIDcol.drop_duplicates(['song']), on='song', how='inner')
inputContent.head(10)

,user_id,song,score,rank,song_id
0,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Superman - Eminem / Dina Rae,0.096418,1,SOHEMBB12A6701E907
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Mockingbird - Eminem,0.089910,2,SOHYSXA12AB0186704
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,U Smile - Justin Bieber,0.061363,3,SOGCWUH12AC90732B0
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm Back - Eminem,0.057932,4,SOLXDDC12A6701FBFD
4,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Here Without You - 3 Doors Down,0.055542,5,SOOJJCT12A6310E1C0
5,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Teach Me How To Dougie - California Swag District,0.054701,6,SOSZJFV12AB01878CB
6,4bd88bfb25263a75bbdd467e74018f4ae570e5df,American Idiot [feat. Green Day & The Cast Of ...,0.054508,7,SODEAWL12AB0187032
7,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Monster - Lady GaGa,0.052736,8,SOEYVHS12AB0181D31
8,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hellbound - J-Black & Masta Ace,0.052564,9,SOLGBMJ12A8C13941A
9,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You Found Me (Album Version) - The Fray,0.052564,10,SOKYJYU12AB0186429


Content-Based Filtering using cosine similarity

In [47]:
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
data1 = pd.read_csv('data.csv')
data1.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [49]:
features_df = data1

In [50]:
scaled_features = MinMaxScaler().fit_transform([
features_df['valence'].values,
features_df['acousticness'].values,
features_df['duration_ms'].values,
features_df['energy'].values,
features_df['liveness'].values,
features_df['speechiness'].values,
features_df['loudness'].values,
features_df['key'].values,
features_df['tempo'].values,
features_df['instrumentalness'],
features_df['explicit'],
features_df['danceability'],
  ])

In [51]:
features_df[['key','acousticness','danceability','duration_ms','energy','explicit','instrumentalness','liveness','loudness','speechiness','tempo','valence']] = scaled_features.T

In [52]:
# features_df = features_df.drop('ArtistName', axis = 1)
# features_df = features_df.drop('Title',axis = 1)
features_df = features_df.drop('year', axis = 1)
features_df = features_df.drop('mode', axis = 1)
features_df = features_df.drop('popularity', axis = 1)
features_df = features_df.drop('release_date', axis = 1)

In [53]:
features_df.head()

,valence,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,name,speechiness,tempo
0,0.000024,0.000025,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",1.0,0.000024,0.000025,0.000024,4BJqT0PrAfrxzMOxytFOIz,0.0,0.000024,0.000036,0.000122,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.000025,0.000024
1,0.000073,0.000073,['Dennis Day'],1.0,0.000071,0.000070,0.000071,7xPhfUan2yNtyFG0cUWkt8,0.0,0.000074,0.000108,0.000406,Clancy Lowered the Boom,0.000069,0.000069
2,0.000030,0.000032,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,1.0,0.000030,0.000030,0.000030,1o6I8BglA6ylDMrIELygv1,0.0,0.000030,0.000036,0.000250,Gati Bali,0.000032,0.000030
3,0.000046,0.000049,['Frank Parker'],1.0,0.000046,0.000046,0.000045,3ftBPsC5vPBKxYSee08FDH,0.0,0.000045,0.000068,0.000521,Danny Boy,0.000044,0.000044
4,0.000063,0.000066,['Phil Regan'],1.0,0.000062,0.000062,0.000061,4d6HGyGT8e121BsdKmw9v6,0.0,0.000062,0.000079,0.000670,When Irish Eyes Are Smiling,0.000061,0.000061


In [54]:
features_df['artists'] = features_df['artists'].str.replace('[', '',regex=True)
features_df['artists'] = features_df['artists'].str.replace(']', '',regex=True)
features_df['artists'] = features_df['artists'].str.replace('\'', '',regex=True)

In [55]:
features_df['song'] = features_df['name']+' - '+features_df['artists']
features_df.head()

,valence,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,name,speechiness,tempo,song
0,0.000024,0.000025,"Sergei Rachmaninoff, James Levine, Berliner Ph...",1.0,0.000024,0.000025,0.000024,4BJqT0PrAfrxzMOxytFOIz,0.0,0.000024,0.000036,0.000122,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.000025,0.000024,"Piano Concerto No. 3 in D Minor, Op. 30: III. ..."
1,0.000073,0.000073,Dennis Day,1.0,0.000071,0.000070,0.000071,7xPhfUan2yNtyFG0cUWkt8,0.0,0.000074,0.000108,0.000406,Clancy Lowered the Boom,0.000069,0.000069,Clancy Lowered the Boom - Dennis Day
2,0.000030,0.000032,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,1.0,0.000030,0.000030,0.000030,1o6I8BglA6ylDMrIELygv1,0.0,0.000030,0.000036,0.000250,Gati Bali,0.000032,0.000030,Gati Bali - KHP Kridhamardawa Karaton Ngayogya...
3,0.000046,0.000049,Frank Parker,1.0,0.000046,0.000046,0.000045,3ftBPsC5vPBKxYSee08FDH,0.0,0.000045,0.000068,0.000521,Danny Boy,0.000044,0.000044,Danny Boy - Frank Parker
4,0.000063,0.000066,Phil Regan,1.0,0.000062,0.000062,0.000061,4d6HGyGT8e121BsdKmw9v6,0.0,0.000062,0.000079,0.000670,When Irish Eyes Are Smiling,0.000061,0.000061,When Irish Eyes Are Smiling - Phil Regan


In [56]:
features_df = features_df.drop('artists', axis = 1)
features_df = features_df.drop('name', axis = 1)

In [57]:
features_df.head()

,valence,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,speechiness,tempo,song
0,0.000024,0.000025,1.0,0.000024,0.000025,0.000024,4BJqT0PrAfrxzMOxytFOIz,0.0,0.000024,0.000036,0.000122,0.000025,0.000024,"Piano Concerto No. 3 in D Minor, Op. 30: III. ..."
1,0.000073,0.000073,1.0,0.000071,0.000070,0.000071,7xPhfUan2yNtyFG0cUWkt8,0.0,0.000074,0.000108,0.000406,0.000069,0.000069,Clancy Lowered the Boom - Dennis Day
2,0.000030,0.000032,1.0,0.000030,0.000030,0.000030,1o6I8BglA6ylDMrIELygv1,0.0,0.000030,0.000036,0.000250,0.000032,0.000030,Gati Bali - KHP Kridhamardawa Karaton Ngayogya...
3,0.000046,0.000049,1.0,0.000046,0.000046,0.000045,3ftBPsC5vPBKxYSee08FDH,0.0,0.000045,0.000068,0.000521,0.000044,0.000044,Danny Boy - Frank Parker
4,0.000063,0.000066,1.0,0.000062,0.000062,0.000061,4d6HGyGT8e121BsdKmw9v6,0.0,0.000062,0.000079,0.000670,0.000061,0.000061,When Irish Eyes Are Smiling - Phil Regan


In [58]:
song_df2 = pd.merge(inputContent, features_df.drop_duplicates(['song']), on='song', how='inner')
song_df2.head(10)

,user_id,song,score,rank,song_id,valence,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,speechiness,tempo
0,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Mockingbird - Eminem,0.089910,2,SOHYSXA12AB0186704,0.000018,0.000016,1.0,0.000018,0.000016,0.000016,561jH07mF1jHuk7KlaeF0s,0.0,0.000016,0.000015,0.000350,0.000015,0.000019
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,U Smile - Justin Bieber,0.061363,3,SOGCWUH12AC90732B0,0.000031,0.000030,1.0,0.000030,0.000028,0.000027,6Xw2FLih8m5ItDbyP992HH,0.0,0.000029,0.000032,0.000599,0.000027,0.000027
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm Back - Eminem,0.057932,4,SOLXDDC12A6701FBFD,0.000016,0.000013,1.0,0.000015,0.000014,0.000013,3xVtc7PB4V3AqAfq4Iahli,0.0,0.000015,0.000048,0.000303,0.000013,0.000016
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Here Without You - 3 Doors Down,0.055542,5,SOOJJCT12A6310E1C0,0.000031,0.000029,1.0,0.000031,0.000029,0.000029,3NLrRZoMF0Lx6zTlYqeIo4,0.0,0.000030,0.000070,0.000632,0.000029,0.000029


In [59]:
song_df2.shape

(4, 18)

In [60]:
import numpy as np



In [61]:
nonplaylist = features_df[~features_df['song'].isin(song_df2['song'].values)]
non=nonplaylist
non.head()

,valence,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,speechiness,tempo,song
0,0.000024,0.000025,1.0,0.000024,0.000025,0.000024,4BJqT0PrAfrxzMOxytFOIz,0.0,0.000024,0.000036,0.000122,0.000025,0.000024,"Piano Concerto No. 3 in D Minor, Op. 30: III. ..."
1,0.000073,0.000073,1.0,0.000071,0.000070,0.000071,7xPhfUan2yNtyFG0cUWkt8,0.0,0.000074,0.000108,0.000406,0.000069,0.000069,Clancy Lowered the Boom - Dennis Day
2,0.000030,0.000032,1.0,0.000030,0.000030,0.000030,1o6I8BglA6ylDMrIELygv1,0.0,0.000030,0.000036,0.000250,0.000032,0.000030,Gati Bali - KHP Kridhamardawa Karaton Ngayogya...
3,0.000046,0.000049,1.0,0.000046,0.000046,0.000045,3ftBPsC5vPBKxYSee08FDH,0.0,0.000045,0.000068,0.000521,0.000044,0.000044,Danny Boy - Frank Parker
4,0.000063,0.000066,1.0,0.000062,0.000062,0.000061,4d6HGyGT8e121BsdKmw9v6,0.0,0.000062,0.000079,0.000670,0.000061,0.000061,When Irish Eyes Are Smiling - Phil Regan


In [62]:
non.shape

(170646, 14)

In [72]:
recom =  list(song_df2["song"])
finalRec = []
for i in recom:
    selected_song = i
   
    # compute cosine similarity between the selected song and all other songs exluding selected songs
    selected_song_vector = song_df2[song_df2['song'] == selected_song][[ 'acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','tempo']].values
    similarity_scores = cosine_similarity(selected_song_vector,non[[ 'acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','tempo']].values).flatten()
    
    # rank similarity scores and recommend top N songs
    N = 2
    top_N_recommendations =non.iloc[similarity_scores.argsort()[::-1][0:N+1]]['song'].tolist()
    finalRec = finalRec + top_N_recommendations

outputValues = pd.DataFrame(finalRec,columns=['Song_Name'])
outputValues = outputValues.style.set_properties(**{'text-align': 'left'})
outputValues = outputValues.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])
display(outputValues)

,Song_Name
0,The Underground in America - Pantera
1,"Say Goodbye to Hollywood - Live at Carnegie Hall, New York, NY - June 1977 - Billy Joel"
2,"You Never Know - Immortal Technique, Jean Grae"
3,Bold as Love - Jimi Hendrix
4,Against All Odds (Take A Look At Me Now) - Mariah Carey
5,Zombies Ate My Neighbors - Single File
6,You're A Customer - EPMD
7,G'D Up - G-Unit
8,The One - Limp Bizkit
9,The New World - X
